In [ ]:
#pip install pandas plotly seaborn matplotlib folium geopandas networkx streamlit pydeck

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import geopandas as gpd
import networkx as nx
import streamlit as st
import pydeck as pdk

# Enable inline plotting for matplotlib
%matplotlib inline

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('olympics_dataset.csv')

# Display the first few rows
df.head()


,player_id,Name,Sex,Team,NOC,Year,Season,City,Sport,Event,Medal
0,0,A Dijiang,M,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,1,A Lamusi,M,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No medal
2,2,Gunnar Aaby,M,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,No medal
3,3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,26,Cornelia (-strannood),F,Netherlands,NED,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,No medal


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252565 entries, 0 to 252564
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   player_id  252565 non-null  int64 
 1   Name       252565 non-null  object
 2   Sex        252565 non-null  object
 3   Team       252565 non-null  object
 4   NOC        252565 non-null  object
 5   Year       252565 non-null  int64 
 6   Season     252565 non-null  object
 7   City       252565 non-null  object
 8   Sport      252565 non-null  object
 9   Event      252565 non-null  object
 10  Medal      252565 non-null  object
dtypes: int64(2), object(9)
memory usage: 21.2+ MB


In [ ]:
missing_values = df.isnull().sum()

print(missing_values)

player_id    0
Name         0
Sex          0
Team         0
NOC          0
Year         0
Season       0
City         0
Sport        0
Event        0
Medal        0
dtype: int64


a. Gender Distributions


In [ ]:
gender_counts = df['Sex'].value_counts().reset_index()
gender_counts.columns = ['Sex', 'Count']

fig = px.pie(gender_counts, names='Sex', values='Count',
             title='Gender Distribution of Athletes',
             color='Sex',
             color_discrete_map={'M':'blue', 'F':'pink', 'Unknown':'gray'})
fig.show()

b. Athletes by Country (Top 10)


In [ ]:
top_countries = df['Team'].value_counts().head(10).reset_index()
top_countries.columns = ['Team', 'Athlete_Count']

fig = px.bar(top_countries, x='Athlete_Count', y='Team', orientation='h',
             title='Top 10 Countries by Athlete Count',
             labels={'Athlete_Count':'Number of Athletes', 'Team':'Country'},
             color='Athlete_Count',
             color_continuous_scale='Viridis')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

c. Total Athletes per Year (Top 10 countries)


In [ ]:
# athletes_per_year = df.groupby('Year')['player_id'].count().reset_index()
# athletes_per_yr_cntry = df.groupby(['Year', 'Team'])['player_id'].count().reset_index()
# athletes_per_yr_cntry.columns = ['Year', 'Team','Athlete_Count']

top10_countries = df['Team'].value_counts().nlargest(10).index
top10_df = df[df['Team'].isin(top10_countries)]
athletes_per_yr_cntry = top10_df.groupby(['Year', 'Team'])['player_id'].count().reset_index()
athletes_per_yr_cntry.columns = ['Year', 'Team','Athlete_Count']

fig = px.line(athletes_per_yr_cntry, x='Year', y='Athlete_Count', color='Team',
              labels={'Athlete_Count':'Number of Athletes'},title='Total Athletes per Year (Top 10 Countries)',
              color_discrete_sequence=["red","blue","yellow","green","pink","orange","purple","magenta","violet","cyan"])
fig.show()

In [ ]:
# Group data by Year and ISO_Code to get Athlete_Count
athletes_per_country_year = df.groupby(['Year', 'NOC']).size().reset_index(name='Athlete_Count')

# Display the aggregated data
athletes_per_country_year.head()


,Year,NOC,Athlete_Count
0,1896,AUS,5
1,1896,AUT,8
2,1896,DEN,15
3,1896,FRA,26
4,1896,GBR,25


In [ ]:
# Create the animated choropleth map
fig = px.choropleth(
    athletes_per_country_year,
    locations='NOC',
    color='Athlete_Count',
    hover_name='NOC',
    animation_frame='Year',
    color_continuous_scale='Blues',
    range_color=(athletes_per_country_year['Athlete_Count'].min(), athletes_per_country_year['Athlete_Count'].max()),
    title='Athletes Distribution by Country Over Time',
    labels={'Athlete_Count':'Number of Athletes'},
    projection='natural earth'
)

# Update geographical layout for better visualization
fig.update_geos(
    showcountries=True,
    showcoastlines=True,
    showland=True,
    fitbounds="locations",
    landcolor="lightgray",
    oceancolor="lightblue"
)

# Update layout for better aesthetics
fig.update_layout(
    coloraxis_colorbar=dict(
        title="Number of Athletes",
        thickness=15,
        len=0.75,
        yanchor="top",
        y=0.95,
        ticks="outside"
    )
)

fig.show()


e.Treemap of Sports and Events
A treemap to show the hierarchical distribution of sports and their respective events based on athlete counts.

f. Aggregate data: Number of athletes per Sport and Event


In [ ]:
sunburst_data = df.groupby(['Sport', 'Event']).size().reset_index(name='Athlete_Count')

fig = px.sunburst(
    sunburst_data,
    path=['Sport', 'Event'],
    values='Athlete_Count',
    color='Athlete_Count',
    color_continuous_scale='RdBu',
    title='Hierarchy of Sports and Events by Athlete Count'
)

fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()

g. Heatmap: Athlete Participation by Gender and Sport

In [ ]:
heatmap_data = df.groupby(['Sex', 'Sport']).size().reset_index(name='Athlete_Count')

fig = px.density_heatmap(
    heatmap_data,
    x='Sex',
    y='Sport',
    z='Athlete_Count',
    histfunc='sum',
    color_continuous_scale='Viridis',
    title='Heatmap: Athlete Participation by Gender and Sport',
    labels={'Athlete_Count': 'Number of Athletes'}
)

fig.update_layout(xaxis_nticks=20)
fig.show()


In [62]:
df = df.dropna(subset=['Year', 'Sport', 'Sex', 'Team', 'NOC'])
df['Year'] = df['Year'].astype(int)

country_counts = df['Team'].value_counts().reset_index()
country_counts.columns = ['Team', 'Athlete_Count']

top_n = 10
top_countries = country_counts.head(top_n)

top_countries_noc = df[df['Team'].isin(top_countries['Team'])][['Team', 'NOC']].drop_duplicates()

top_countries = top_countries.merge(top_countries_noc, on='Team', how='left')

print("Unique NOC Codes in Top Countries:", top_countries['NOC'].unique())

surface_data = df.groupby(['Sport', 'Year']).size().reset_index(name='Athlete_Count')

surface_pivot = surface_data.pivot(index='Sport', columns='Year', values='Athlete_Count').fillna(0)

surface_pivot = surface_pivot.sort_values(by=surface_pivot.columns.tolist(), ascending=False)

# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[
        [{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
        [None, {"type": "surface"}]
    ],
    subplot_titles=(
        "Geographical Distribution of Athletes",
        "Top 10 Countries by Athlete Count",
        "Athlete Participation Across Sports and Years"
    )
)

# Add Scattergeo map of athlete locations using NOC codes
fig.add_trace(
    go.Scattergeo(
        locations=top_countries['NOC'],
        locationmode='ISO-3',  # Ensures that NOC codes are interpreted as ISO Alpha-3 codes
        text=top_countries['Team'] + ': ' + top_countries['Athlete_Count'].astype(str) + ' Athletes',
        mode="markers",
        marker=dict(
            color="crimson",
            size=10,
            opacity=0.8
        ),
        hoverinfo="text"
    ),
    row=1, col=1
)

# Add bar chart of top 10 countries
fig.add_trace(
    go.Bar(
        x=top_countries['Athlete_Count'][::-1],  # Reverse for descending order
        y=top_countries['Team'][::-1],
        orientation='h',
        marker=dict(color="crimson"),
        hoverinfo='x',
        showlegend=False
    ),
    row=1, col=2
)

# Add 3D surface of athlete participation
fig.add_trace(
    go.Surface(
        z=surface_pivot.values,
        x=surface_pivot.columns.astype(int).tolist(),
        y=surface_pivot.index.tolist(),
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Number of Athletes'),
        opacity=0.8
    ),
    row=2, col=2
)

# Update geo subplot properties
fig.update_geos(
    projection_type="natural earth",
    showcountries=True,
    showcoastlines=True,
    showland=True,
    landcolor="lightgray",
    oceancolor="LightBlue",
    countrycolor="white",
    showocean=True
)

fig.update_xaxes(tickangle=45, row=1, col=2)

fig.update_layout(
    template="plotly_dark",
    margin=dict(r=10, t=100, b=40, l=60),
    height=800,
    title_text="Olympic Athletes Dashboard: Geographical Distribution, Top Countries, and Participation Trends",
    annotations=[
        dict(
            text="Source: Olympics Dataset",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0
        )
    ]
)

fig.show()


   player_id                   Name Sex            Team  NOC  Year  Season  \
0          0              A Dijiang   M           China  CHN  1992  Summer   
1          1               A Lamusi   M           China  CHN  2012  Summer   
2          2            Gunnar Aaby   M         Denmark  DEN  1920  Summer   
3          3            Edgar Aabye   M  Denmark/Sweden  DEN  1900  Summer   
4         26  Cornelia (-strannood)   F     Netherlands  NED  1932  Summer   

          City       Sport                         Event     Medal  
0    Barcelona  Basketball   Basketball Men's Basketball  No medal  
1       London        Judo  Judo Men's Extra-Lightweight  No medal  
2    Antwerpen    Football       Football Men's Football  No medal  
3        Paris  Tug-Of-War   Tug-Of-War Men's Tug-Of-War      Gold  
4  Los Angeles   Athletics  Athletics Women's 100 metres  No medal  
Unique NOC Codes in Top Countries: ['USA' 'GBR' 'FRA' 'ITA' 'GER' 'AUS' 'CAN' 'JPN' 'HUN' 'SWE']


In [112]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# -----------------------------------
# 1. Load and Prepare the Dataset
# -----------------------------------

# Load the Olympic athletes dataset
df = pd.read_csv('olympics_dataset.csv')

# Data Cleaning: Drop rows with missing critical information
df = df.dropna(subset=['Year', 'Sport', 'Sex', 'Team', 'NOC'])
df['Year'] = df['Year'].astype(int)

# Aggregate data: Count of athletes per Team (Country)
country_counts = df['Team'].value_counts().reset_index()
country_counts.columns = ['Team', 'Athlete_Count']

# Select top 10 countries for clarity in visualizations
top_n = 10
top_countries = country_counts.head(top_n)

# Extract unique NOC codes for the top countries
top_countries_noc = df[df['Team'].isin(top_countries['Team'])][['Team', 'NOC']].drop_duplicates()

# Merge NOC codes with top_countries
top_countries = top_countries.merge(top_countries_noc, on='Team', how='left')

# Verify unique NOC codes in top_countries
print("Unique NOC Codes in Top Countries:", top_countries['NOC'].unique())

# Aggregate data: Number of athletes per Sport and Year
surface_data = df.groupby(['Sport', 'Year']).size().reset_index(name='Athlete_Count')

# Create a pivot table with Sports as rows and Years as columns
surface_pivot = surface_data.pivot(index='Sport', columns='Year', values='Athlete_Count').fillna(0)

# Sort sports by total athlete count for better visualization
surface_pivot = surface_pivot.sort_values(by=surface_pivot.columns.tolist(), ascending=False)

# -----------------------------------
# 2. Initialize Subplots
# -----------------------------------

# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[
        [{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
        [None, {"type": "surface"}]
    ],
    subplot_titles=(
        "Geographical Distribution of Athletes",
        "Top 10 Countries by Athlete Count",
        "Athlete Participation Across Sports and Years"
    ),
    horizontal_spacing=0.05,  # Reduced spacing between columns
    vertical_spacing=0.05     # Reduced spacing between rows
)

# -----------------------------------
# 3. Add Scattergeo Plot (Simulated 3D Globe)
# -----------------------------------

# Add Scattergeo map of athlete locations using NOC codes
scattergeo_trace = go.Scattergeo(
    locations=top_countries['NOC'],
    locationmode='ISO-3',  # Ensures that NOC codes are interpreted as ISO Alpha-3 codes
    text=top_countries['Team'] + ': ' + top_countries['Athlete_Count'].astype(str) + ' Athletes',
    mode="markers",
    marker=dict(
        color="crimson",
        size=top_countries['Athlete_Count'] / top_countries['Athlete_Count'].max() * 30,  # Scale marker sizes
        opacity=0.8
    ),
    hoverinfo="text"
)

fig.add_trace(
    scattergeo_trace,
    row=1, col=1
)

# -----------------------------------
# 4. Define Frames for Automatic Rotation
# -----------------------------------

# Define the number of frames and the step of rotation
num_frames = 72  # 72 frames for 360/5 degrees rotation steps
rotation_step = 5  # degrees per frame

frames = []

for i in range(num_frames):
    rotation = rotation_step * i
    frame = go.Frame(
        data=[
            go.Scattergeo(
                locations=top_countries['NOC'],
                locationmode='ISO-3',
                text=top_countries['Team'] + ': ' + top_countries['Athlete_Count'].astype(str) + ' Athletes',
                mode="markers",
                marker=dict(
                    color="crimson",
                    size=top_countries['Athlete_Count'] / top_countries['Athlete_Count'].max() * 30,
                    opacity=0.8
                ),
                hoverinfo="text"
            )
        ],
        layout=go.Layout(
            geo=dict(
                projection_rotation=dict(lon=rotation, lat=0, roll=0)
            )
        )
    )
    frames.append(frame)

# Add frames to the figure
fig.frames = frames

# -----------------------------------
# 5. Update Geos for Initial Layout
# -----------------------------------

# Update the Scattergeo projection to 'orthographic' for a globe-like appearance
fig.update_geos(
    projection_type="orthographic",
    showcountries=True,
    showcoastlines=True,
    showland=True,
    landcolor="lightgray",
    oceancolor="Darkblue",
    countrycolor="white",
    showocean=True,
    resolution=50,
    projection_rotation=dict(lon=0, lat=0, roll=0)  # Initial rotation
)

# -----------------------------------
# 6. Add Animation Controls
# -----------------------------------

# Add updatemenus for Play and Pause buttons
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[
                         None,
                         dict(frame=dict(duration=50, redraw=True),
                              transition=dict(duration=0),
                              fromcurrent=True,
                              mode='immediate')
                     ]),
                dict(label="Pause",
                     method="animate",
                     args=[
                         [None],
                         dict(frame=dict(duration=0, redraw=False),
                              transition=dict(duration=0),
                              mode='immediate')
                     ])
            ],
            direction="left",
            pad={"r": 10, "t": 87},
            showactive=False,
            x=0.1,
            xanchor="right",
            y=0,
            yanchor="top"
        )
    ]
)

# -----------------------------------
# 7. Add Bar Chart
# -----------------------------------

# Add bar chart of top 10 countries
fig.add_trace(
    go.Bar(
        x=top_countries['Athlete_Count'][::-1],  # Reverse for descending order
        y=top_countries['Team'][::-1],
        orientation='h',
        marker=dict(color="crimson"),
        hoverinfo='x',
        showlegend=False
    ),
    row=1, col=2
)

# Rotate x-axis labels for the bar chart for better readability
fig.update_xaxes(tickangle=0, row=1, col=2)
# fig.update_yaxes(title_text="Countries", row=1, col=2)

# -----------------------------------
# 8. Add 3D Surface Plot
# -----------------------------------

# Add 3D surface of athlete participation
fig.add_trace(
    go.Surface(
        z=surface_pivot.values,
        x=surface_pivot.columns.astype(int).tolist(),
        y=surface_pivot.index.tolist(),
        colorscale='Cividis',
        showscale=True,
        colorbar=dict(title='Number of Athletes'),
        opacity=0.8
    ),
    row=2, col=2
)

# -----------------------------------
# 9. Final Layout Adjustments
# -----------------------------------

# Update layout settings
fig.update_layout(
    template="plotly_dark",
    margin=dict(r=20, t=150, b=60, l=80),
    height=900,  # Increased height for better visibility
    width=1200,  # Increased width for better visibility
    title_text="Olympic Athletes Dashboard: Geographical Distribution, Top Countries, and Participation Trends",
    annotations=[
        dict(
            text="Source: Olympics Dataset",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0
        )
    ],
    hovermode='closest',
    title_font_size=24,
    title_x=0.5  # Center the title
)

# -----------------------------------
# 10. Display the Figure
# -----------------------------------

fig.show()


Unique NOC Codes in Top Countries: ['USA' 'GBR' 'FRA' 'ITA' 'GER' 'AUS' 'CAN' 'JPN' 'HUN' 'SWE']


In [100]:
df_medals = df[df['Medal'].notnull()]

# Aggregate medal counts by Year and NOC
medal_counts = df_medals.groupby(['Year', 'NOC']).size().reset_index(name='Medal_Count')

# Create animated bar chart
fig = px.bar(medal_counts,
             x='NOC',
             y='Medal_Count',
             color='NOC',
             animation_frame='Year',
             range_y=[0, medal_counts['Medal_Count'].max() + 50],
             title='Medal Counts by Country Over Years',
             labels={'NOC':'Country', 'Medal_Count':'Number of Medals'},
             color_discrete_sequence=["red","blue","yellow","green","pink","orange","purple","magenta","violet","cyan"])

fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


In [105]:
import plotly.express as px

# Aggregate medal types
medal_tally = df_medals.groupby(['NOC', 'Medal']).size().reset_index(name='Count')

# Create bubble chart
fig = px.scatter(medal_tally,
                 x='NOC',
                 y='Count',
                 size='Count',
                 color='Medal',
                 hover_name='NOC',
                 title='Medal Tally by Country and Medal Type',
                 labels={'NOC': 'Country', 'Count': 'Number of Medals'},
                 size_max=60,
                color_discrete_sequence=["orange","blue","silver","gold"])


fig.show()


In [111]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Assuming df_medals is already defined and loaded
# If not, you can load it using:
# df = pd.read_csv('olympics_dataset.csv')
# df_medals = df[df['Medal'].notnull()]

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Medal Counts by Year", "Top Countries", "Medals by Sport", "3D Medal Distribution"),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "heatmap"}, {"type": "scatter3d"}]]
)

# ----------------------------
# 1. Bar Chart: Medal Counts by Year
# ----------------------------
medal_counts = df_medals.groupby('Year').size().reset_index(name='Medal_Count')

# Define a color scale for the bar chart
bar_colors = px.colors.qualitative.Plotly  # You can choose other color scales from Plotly

fig.add_trace(
    go.Bar(
        x=medal_counts['Year'],
        y=medal_counts['Medal_Count'],
        name='Medals by Year',
        marker=dict(
            color=medal_counts['Medal_Count'],  # Color based on Medal_Count
            colorscale='Blues',  # Choose a colorscale: 'Viridis', 'Cividis', 'Blues', etc.
            showscale=True,  # Show color scale
            colorbar=dict(title="Medal Count")
        )
    ),
    row=1, col=1
)

# ----------------------------
# 2. Pie Chart: Top 10 Countries by Medal Count
# ----------------------------
top_countries = df_medals['NOC'].value_counts().nlargest(10)

# Define a color palette for the pie chart
pie_colors = px.colors.qualitative.Set3  # You can choose other color palettes like 'Pastel', 'D3', etc.

fig.add_trace(
    go.Pie(
        labels=top_countries.index,
        values=top_countries.values,
        name='Top Countries',
        marker=dict(
            colors=pie_colors  # Assign colors to each slice
        ),
        textinfo='percent+label',  # Display both percentage and label
        hoverinfo='label+value+percent'
    ),
    row=1, col=2
)

# ----------------------------
# 3. Heatmap: Medals by Sport and Country
# ----------------------------
heatmap_data = df_medals.pivot_table(index='Sport', columns='NOC', values='Medal', aggfunc='count', fill_value=0)
fig.add_trace(
    go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='Blues',
        colorbar=dict(title="Medal Count")
    ),
    row=2, col=1
)

# ----------------------------
# 4. 3D Scatter: Medals, Athletes, and Sports
# ----------------------------
medal_sport = df_medals.groupby(['Sport', 'Year']).size().reset_index(name='Medal_Count')

fig.add_trace(
    go.Scatter3d(
        x=medal_sport['Sport'],
        y=medal_sport['Year'],
        z=medal_sport['Medal_Count'],
        mode='markers',
        marker=dict(
            size=5,
            color=medal_sport['Medal_Count'],  # Color based on Medal_Count
            colorscale='Viridis',  # Choose a colorscale: 'Viridis', 'Cividis', etc.
            opacity=0.8,
            colorbar=dict(title="Medal Count")
        ),
        text=medal_sport['Sport'],
        name='Medals, Athletes, and Sports'
    ),
    row=2, col=2
)

# ----------------------------
# Final Layout Adjustments
# ----------------------------
fig.update_layout(
    height=800,
    width=1200,
    title_text="Olympic Medals Dashboard",
    showlegend=False,  # Hide legend for clarity; adjust as needed
    template='plotly_white'  # Optional: use a clean template
)

# Display the figure
fig.show()
